Installing the required libraries.
1. Huggingface's datasets library to load the Wikipedia dataset.
2. The sentence_transformers library for access to models we'll use to embed our documents.
3. ChromaDB for storing our documents and associated embeddings, and for retrieving documents similar to our input query.



In [ ]:
!pip install -q datasets -U sentence_transformers chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wh

Let's define a function to allow use accelerated computing with a GPU should on exist on the sytem.

In [ ]:
import torch

def try_gpu():
  return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Downloading Dataset from the Hub

In [ ]:
# Download dataset from the hub
from datasets import load_dataset, Dataset

docs = load_dataset(f"Cohere/wikipedia-22-12-simple-embeddings", split="train")

# Extract only columns [ids, title, text]
ids, titles, passages = docs["id"], docs["title"], docs["text"]

In [ ]:
len(ids), len(passages), len(ids)==len(passages)

(485859, 485859, True)

Download sentence transformer model.
We'll be using the multi-qa-mpnet-base-dot-v1 model

In [ ]:
# Using sentence transformers
from sentence_transformers import SentenceTransformer, util

device = try_gpu()
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1').to(device)

Generate embeddings for all passages in the dataset

In [ ]:
# Create corpus embeddings
#start=0
#end=100000
corpus_embds = model.encode(passages[start:end], batch_size=256, show_progress_bar=True)
corpus_embds = torch.from_numpy(corpus_embds)

Batches:   0%|          | 0/391 [00:00<?, ?it/s]

Now on to saving embeddings for future retrieval.
1. Import chromadb and embedding_functions to specify which sentence_transformer model to use.
2. Create client.
3 Specify embedding function (model) to use.

In [ ]:
import os
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions

ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="multi-qa-mpnet-base-dot-v1")

In [ ]:
def check_connection(client, db):
    cols = client.list_collections()
    if len(cols) == 0:
      return False
    col_name = cols[0].dict().get("name")

    # CHECK if nsmq_ai db exists in collection
    if col_name==db:
        return True

    return False

In [ ]:
def create_db(client, db, docs, ids, embds=None):
  # Create a new db if db does not exist, else get existing db and update it
  db_exists = check_connection(client, db)

  if db_exists:
    collection = client.get_collection(name=db, embedding_function=ef)
  else:
    collection = client.create_collection(name=db, metadata={"hnsw:space":"cosine"}, embedding_function=ef)

  ids_as_strings = [str(id) for id in ids]

  print("Embeddings generation will take some time depending on number of documents. Processing...")

  if embds is not None:
    collection.add(
        documents=docs,
        embeddings=embds.tolist(),
        ids=ids_as_strings
    )
  else:
    collection.add(
        documents=docs,
        ids=ids_as_strings
    )

  print(f"Total docs stored in db: {collection.count()}")

  client.persist()

In [ ]:
def query_db(query, n_results=1):
  results = collection.query(
      query_texts=[query],
      n_results=n_results
  )

  docs, scores = results["documents"], results["distances"]
  context = ""
  for doc in docs[0]:
    context += doc + " | "
  scores = round((sum(scores[0]))/n_results, 2)

  return context, scores

In [ ]:
# Get access to google drive to store .parquet file in
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# SETUP DATABASE
db = "<DB NAME>"
db_path = "<PATH/TO/DB" # DB Storage path
path_to_db = os.path.join(db_path, "Database")
client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory=path_to_db))

db_exists = check_connection(client, db)

if db_exists:
    collection = client.get_collection(name=db, embedding_function=ef)
else:
    if not os.path.isdir(path_to_db): os.mkdir(path_to_db)
    create_db(client, db, docs=passages[:10], ids=ids[:10], embds=corpus_embds[:10])
    collection = client.get_collection(name=db, embedding_function=ef)

In [ ]:
collection.count()

10

In [ ]:
%timeit
from tqdm import tqdm

step = 10000
for x in tqdm(range(10, len(ids), step)):
  start = x
  end = x + step
  corpus_embds = model.encode(passages[start:end], batch_size=128, show_progress_bar=True)
  corpus_embds = torch.from_numpy(corpus_embds)
  create_db(client, db, passages[start:end], ids[start:end], corpus_embds)

  0%|          | 0/49 [00:00<?, ?it/s]

Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Embeddings generation will take some time depending on number of documents. Processing...
Total docs stored in db: 10010


  2%|▏         | 1/49 [01:23<1:06:44, 83.43s/it]

Batches:   0%|          | 0/79 [00:00<?, ?it/s]

...

 98%|█████████▊| 48/49 [1:29:22<02:07, 127.59s/it]

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

Embeddings generation will take some time depending on number of documents. Processing...
Total docs stored in db: 485859


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

100%|██████████| 49/49 [1:31:11<00:00, 111.67s/it]
